In [1]:
import os
import random
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from openpyxl.workbook import Workbook

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\koenm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_json(r'C:\Users\koenm\OneDrive\Bureaublad\Thesis\output\use\metacritic_final.json')
data.columns

Index(['url', 'title', 'artist', 'datePublished', 'summary', 'meta_score',
       'user_score', 'genre', 'critic-reviews', 'user_reviews'],
      dtype='object')

In [3]:
#checking row
data['user_reviews'][1]

[{'score': '6',
  'reviewer': 'Dalmx',
  'date_published': 'Mar 23, 2021',
  'review_body': 'qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq',
  'helpul_count': '0'},
 {'score': '9',
  'reviewer': 'Menios',
  'date_published': 'Mar  5, 2021',
  'review_body': "YES YES YES! Go for a walk while listening to this album right about now. You're going to feel so much better in these harsh times we live in.",
  'helpul_count': '0'},
 {'score': '10',
  'reviewer': 'ConnorBruce',
  'date_published': 'Feb 20, 2021',
  'review_body': 'really nice makes you think about your life. Hard drive is 100% the peak of the album and a strong contender for SOTY',
  'helpul_count': '0'},
 {'score': '9',
  'reviewer': 'IAmDaan',
  'date_published': 'Feb 25, 2021',
  'review_body': 'Second album "An Overview On Phenomenal Nature" by US songwriter Cassandra Jenkins goes through the expansion stages, acquiring many images and references, monotonous dialogues turning into a mixture of 

In [4]:
finaldf = pd.DataFrame(columns=['url', 'user_reviews_index', 'userscore', 'user_reviewer', 'user_date_published', 'user_review_body'])
for index, value in data.iterrows():
    url = value.url
    userreview = value['user_reviews']
    count = 0
    #print(index, '/', len(data))
    for item in userreview:
        count = count +1
        score = item['score']
        reviewer = item['reviewer']
        pubdate = item['date_published']
        reviewbody = item['review_body']
        helpfulcount = item['helpul_count']
        row = {'url':[url], 'user_reviews_index':[count] ,'userscore': [score],'user_reviewer': [reviewer],'user_date_published': [pubdate],'user_review_body': [reviewbody]}
        rowdf = pd.DataFrame(row)
        finaldf = pd.concat([finaldf, rowdf])

In [6]:
userreviews = finaldf

In [10]:
#save as new file
userreviews.to_excel(r'C:\Users\koenm\OneDrive\Bureaublad\Thesis\output\cleaned\userreviewsexcel.xlsx')

In [11]:
#Drop NAN
userreviews = userreviews[userreviews['user_review_body'].notna()]
data.count()

url               12487
title             12487
artist            12487
datePublished     12486
summary           12340
meta_score        12485
user_score        12487
genre             12416
critic-reviews    12487
user_reviews      12487
dtype: int64

In [12]:
#Delete \n
userreviews['user_review_body'] = userreviews['user_review_body'].str.replace('\n',' ')

In [13]:
#Delete \r
userreviews['user_review_body'] = userreviews['user_review_body'].str.replace('\r',' ')

In [14]:
#Lower case
userreviews['user_review_body'] = userreviews['user_review_body'].str.replace('\r',' ').str.lower()

In [15]:
#check if all is still fine
print(userreviews['user_review_body'])

0    this album is amazing . one of the bests albun...
0    excellent album with enough ear-worms througho...
0    i checked this out due to comparisons to norah...
0    this soul/blues/rock album does just enough to...
0    decided to listen to this album on account of ...
                           ...                        
0    what else can you say about this album? - a ma...
0    iconic, i don't even need to say anything, thi...
0    this review contains spoilers, click expand to...
0    this review contains spoilers, click expand to...
0    teenage dream , how to describe an album that ...
Name: user_review_body, Length: 217403, dtype: object


In [16]:
#Remove numbers
userreviews['user_review_body'] = userreviews['user_review_body'].str.replace('\d+', '')

<ipython-input-16-1903c34ce990>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  userreviews['user_review_body'] = userreviews['user_review_body'].str.replace('\d+', '')


In [17]:
#Remove stopwords
stop = stopwords.words('english')
userreviews['user_review_body'] = userreviews['user_review_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [18]:
#remove punctuation
userreviews['user_review_body'] = userreviews['user_review_body'].str.replace('[^\w\s]', '')

<ipython-input-18-64cbe4fc563a>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  userreviews['user_review_body'] = userreviews['user_review_body'].str.replace('[^\w\s]', '')


In [19]:
#check information to drop for sentiment analysis
userreviews

,url,user_reviews_index,userscore,user_reviewer,user_date_published,user_review_body
0,https://www.metacritic.com/music/the-order-of-...,1,10,popstyle,"Mar 10, 2017",album amazing one bests albuns without doubtt...
0,https://www.metacritic.com/music/the-order-of-...,2,8,jethibea,"Jun 8, 2017",excellent album enough earworms throughout war...
0,https://www.metacritic.com/music/the-order-of-...,3,5,jake_gibbs,"Jun 24, 2017",checked due comparisons norah jones whose musi...
0,https://www.metacritic.com/music/the-order-of-...,4,7,Music_Observer,"May 16, 2017",soulbluesrock album enough maintain solidity c...
0,https://www.metacritic.com/music/the-order-of-...,5,10,SlowRocket,"May 6, 2017",decided listen album account metascore simply ...
...,...,...,...,...,...,...
0,https://www.metacritic.com/music/teenage-dream...,369,8,Vanyagranderia,"Mar 29, 2021",else say album masterpiece
0,https://www.metacritic.com/music/teenage-dream...,370,10,Katyperry12,"Apr 2, 2021",iconic even need say anything album biggest hi...
0,https://www.metacritic.com/music/teenage-dream...,371,10,Dilann,"Apr 9, 2021",review contains spoilers click expand view bes...
0,https://www.metacritic.com/music/teenage-dream...,372,0,JoshMedinna,"Apr 21, 2021",review contains spoilers click expand view tra...


In [20]:
#drop columns not needed for sentiment analysis
userreviewsclean = userreviews.drop(columns=['user_reviews_index','userscore', 'user_reviewer', 'user_date_published'])
userreviewsclean

,url,user_review_body
0,https://www.metacritic.com/music/the-order-of-...,album amazing one bests albuns without doubtt...
0,https://www.metacritic.com/music/the-order-of-...,excellent album enough earworms throughout war...
0,https://www.metacritic.com/music/the-order-of-...,checked due comparisons norah jones whose musi...
0,https://www.metacritic.com/music/the-order-of-...,soulbluesrock album enough maintain solidity c...
0,https://www.metacritic.com/music/the-order-of-...,decided listen album account metascore simply ...
...,...,...
0,https://www.metacritic.com/music/teenage-dream...,else say album masterpiece
0,https://www.metacritic.com/music/teenage-dream...,iconic even need say anything album biggest hi...
0,https://www.metacritic.com/music/teenage-dream...,review contains spoilers click expand view bes...
0,https://www.metacritic.com/music/teenage-dream...,review contains spoilers click expand view tra...


In [21]:
#save to new file
userreviewsclean.to_csv(r'C:\Users\koenm\OneDrive\Bureaublad\Thesis\output\cleaned\usercleanforsentiment.csv')